In [10]:
import pandas as pd
import numpy as np

# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

In [11]:
import re

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()


data = [preprocess_text(t) for t in raw_data]

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

In [18]:
import sqlite3

# Открываем SQLite базу данных
conn = sqlite3.connect('tweet.db')
c = conn.cursor()

with open('tweets.txt', 'w', encoding='utf-8') as f:
    # Считываем тексты твитов 
    i = 0
    for row in c.execute('SELECT ttext FROM sentiment'):
        i += 1
        if row[0]:
            tweet = preprocess_text(row[0])
            # Записываем предобработанные твиты в файл
            print(tweet, file=f)
        if i % 200000 == 0:
            print(i)

200000
400000
600000
800000
1000000
1200000
1400000
1600000
1800000
2000000
2200000
2400000
2600000
2800000
3000000
3200000
3400000
3600000
3800000
4000000
4200000
4400000
4600000
4800000
5000000
5200000
5400000
5600000
5800000
6000000
6200000
6400000
6600000
6800000
7000000
7200000
7400000
7600000
7800000
8000000
8200000
8400000
8600000
8800000
9000000
9200000
9400000
9600000
9800000
10000000
10200000
10400000
10600000
10800000
11000000
11200000
11400000
11600000
11800000
12000000
12200000
12400000
12600000
12800000
13000000
13200000
13400000
13600000
13800000
14000000
14200000
14400000
14600000
14800000
15000000
15200000
15400000
15600000
15800000
16000000
16200000
16400000
16600000
16800000
17000000
17200000
17400000
17600000


In [20]:
import logging
import multiprocessing
import gensim
from gensim.models import Word2Vec

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Считываем файл с предобработанными твитами
data = gensim.models.word2vec.LineSentence('tweets.txt')
# Обучаем модель 
model = Word2Vec(data, size=200, window=5, min_count=3, workers=multiprocessing.cpu_count())
model.save("models/w2v/model.w2v")

2019-07-06 10:58:37,950 : INFO : collecting all words and their counts
2019-07-06 10:58:37,951 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-06 10:58:38,001 : INFO : PROGRESS: at sentence #10000, processed 98345 words, keeping 24248 word types
2019-07-06 10:58:38,044 : INFO : PROGRESS: at sentence #20000, processed 194782 words, keeping 38847 word types
2019-07-06 10:58:38,088 : INFO : PROGRESS: at sentence #30000, processed 290743 words, keeping 50531 word types
2019-07-06 10:58:38,132 : INFO : PROGRESS: at sentence #40000, processed 386269 words, keeping 60653 word types
2019-07-06 10:58:38,176 : INFO : PROGRESS: at sentence #50000, processed 482755 words, keeping 69689 word types
2019-07-06 10:58:38,226 : INFO : PROGRESS: at sentence #60000, processed 578104 words, keeping 78235 word types
2019-07-06 10:58:38,270 : INFO : PROGRESS: at sentence #70000, processed 672686 words, keeping 85840 word types
2019-07-06 10:58:38,320 : INFO : PROGRESS: at s

2019-07-06 10:58:41,265 : INFO : PROGRESS: at sentence #720000, processed 6856226 words, keeping 329362 word types
2019-07-06 10:58:41,315 : INFO : PROGRESS: at sentence #730000, processed 6953689 words, keeping 331913 word types
2019-07-06 10:58:41,361 : INFO : PROGRESS: at sentence #740000, processed 7054203 words, keeping 334539 word types
2019-07-06 10:58:41,405 : INFO : PROGRESS: at sentence #750000, processed 7153523 words, keeping 336844 word types
2019-07-06 10:58:41,451 : INFO : PROGRESS: at sentence #760000, processed 7253230 words, keeping 339124 word types
2019-07-06 10:58:41,496 : INFO : PROGRESS: at sentence #770000, processed 7352424 words, keeping 341418 word types
2019-07-06 10:58:41,540 : INFO : PROGRESS: at sentence #780000, processed 7450987 words, keeping 343877 word types
2019-07-06 10:58:41,586 : INFO : PROGRESS: at sentence #790000, processed 7545233 words, keeping 346595 word types
2019-07-06 10:58:41,629 : INFO : PROGRESS: at sentence #800000, processed 763865

2019-07-06 10:58:44,537 : INFO : PROGRESS: at sentence #1430000, processed 13698594 words, keeping 494012 word types
2019-07-06 10:58:44,585 : INFO : PROGRESS: at sentence #1440000, processed 13799436 words, keeping 496301 word types
2019-07-06 10:58:44,633 : INFO : PROGRESS: at sentence #1450000, processed 13898908 words, keeping 498556 word types
2019-07-06 10:58:44,680 : INFO : PROGRESS: at sentence #1460000, processed 13996483 words, keeping 500687 word types
2019-07-06 10:58:44,728 : INFO : PROGRESS: at sentence #1470000, processed 14094360 words, keeping 502819 word types
2019-07-06 10:58:44,775 : INFO : PROGRESS: at sentence #1480000, processed 14192679 words, keeping 504855 word types
2019-07-06 10:58:44,824 : INFO : PROGRESS: at sentence #1490000, processed 14291910 words, keeping 507157 word types
2019-07-06 10:58:44,871 : INFO : PROGRESS: at sentence #1500000, processed 14391159 words, keeping 509297 word types
2019-07-06 10:58:44,919 : INFO : PROGRESS: at sentence #1510000,

2019-07-06 10:58:47,917 : INFO : PROGRESS: at sentence #2140000, processed 20623466 words, keeping 625917 word types
2019-07-06 10:58:47,966 : INFO : PROGRESS: at sentence #2150000, processed 20722248 words, keeping 627462 word types
2019-07-06 10:58:48,014 : INFO : PROGRESS: at sentence #2160000, processed 20819863 words, keeping 629122 word types
2019-07-06 10:58:48,062 : INFO : PROGRESS: at sentence #2170000, processed 20917554 words, keeping 630708 word types
2019-07-06 10:58:48,110 : INFO : PROGRESS: at sentence #2180000, processed 21015641 words, keeping 632393 word types
2019-07-06 10:58:48,159 : INFO : PROGRESS: at sentence #2190000, processed 21113827 words, keeping 634124 word types
2019-07-06 10:58:48,206 : INFO : PROGRESS: at sentence #2200000, processed 21212464 words, keeping 635844 word types
2019-07-06 10:58:48,255 : INFO : PROGRESS: at sentence #2210000, processed 21310897 words, keeping 637585 word types
2019-07-06 10:58:48,304 : INFO : PROGRESS: at sentence #2220000,

2019-07-06 10:58:51,362 : INFO : PROGRESS: at sentence #2850000, processed 27587700 words, keeping 742350 word types
2019-07-06 10:58:51,412 : INFO : PROGRESS: at sentence #2860000, processed 27686628 words, keeping 744033 word types
2019-07-06 10:58:51,462 : INFO : PROGRESS: at sentence #2870000, processed 27788789 words, keeping 745708 word types
2019-07-06 10:58:51,509 : INFO : PROGRESS: at sentence #2880000, processed 27885614 words, keeping 747258 word types
2019-07-06 10:58:51,555 : INFO : PROGRESS: at sentence #2890000, processed 27979645 words, keeping 748829 word types
2019-07-06 10:58:51,601 : INFO : PROGRESS: at sentence #2900000, processed 28075614 words, keeping 750449 word types
2019-07-06 10:58:51,648 : INFO : PROGRESS: at sentence #2910000, processed 28172950 words, keeping 752004 word types
2019-07-06 10:58:51,694 : INFO : PROGRESS: at sentence #2920000, processed 28270410 words, keeping 753472 word types
2019-07-06 10:58:51,741 : INFO : PROGRESS: at sentence #2930000,

...

2019-07-06 11:12:47,886 : INFO : EPOCH 4 - PROGRESS: at 93.70% examples, 733470 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:12:48,895 : INFO : EPOCH 4 - PROGRESS: at 94.25% examples, 733779 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:12:49,909 : INFO : EPOCH 4 - PROGRESS: at 94.78% examples, 734145 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:12:50,933 : INFO : EPOCH 4 - PROGRESS: at 95.19% examples, 733579 words/s, in_qsize 22, out_qsize 0
2019-07-06 11:12:51,944 : INFO : EPOCH 4 - PROGRESS: at 95.77% examples, 734453 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:12:52,962 : INFO : EPOCH 4 - PROGRESS: at 96.27% examples, 734329 words/s, in_qsize 0, out_qsize 1
2019-07-06 11:12:53,963 : INFO : EPOCH 4 - PROGRESS: at 96.72% examples, 733984 words/s, in_qsize 19, out_qsize 0
2019-07-06 11:12:54,968 : INFO : EPOCH 4 - PROGRESS: at 97.30% examples, 734729 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:12:55,975 : INFO : EPOCH 4 - PROGRESS: at 97.76% examples, 734530 words/s, in_qs

2019-07-06 11:13:50,872 : INFO : EPOCH 5 - PROGRESS: at 26.65% examples, 733898 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:13:51,878 : INFO : EPOCH 5 - PROGRESS: at 27.17% examples, 734341 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:13:52,895 : INFO : EPOCH 5 - PROGRESS: at 27.69% examples, 733925 words/s, in_qsize 5, out_qsize 2
2019-07-06 11:13:53,897 : INFO : EPOCH 5 - PROGRESS: at 28.24% examples, 734478 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:13:54,917 : INFO : EPOCH 5 - PROGRESS: at 28.72% examples, 733360 words/s, in_qsize 7, out_qsize 7
2019-07-06 11:13:55,922 : INFO : EPOCH 5 - PROGRESS: at 29.22% examples, 733484 words/s, in_qsize 17, out_qsize 0
2019-07-06 11:13:56,929 : INFO : EPOCH 5 - PROGRESS: at 29.82% examples, 735578 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:13:57,929 : INFO : EPOCH 5 - PROGRESS: at 30.34% examples, 735936 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:13:58,932 : INFO : EPOCH 5 - PROGRESS: at 30.79% examples, 734514 words/s, in_qsi

2019-07-06 11:15:04,689 : INFO : EPOCH 5 - PROGRESS: at 65.62% examples, 730193 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:15:05,690 : INFO : EPOCH 5 - PROGRESS: at 66.05% examples, 729255 words/s, in_qsize 18, out_qsize 1
2019-07-06 11:15:06,695 : INFO : EPOCH 5 - PROGRESS: at 66.67% examples, 730247 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:15:07,709 : INFO : EPOCH 5 - PROGRESS: at 67.15% examples, 729789 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:15:08,719 : INFO : EPOCH 5 - PROGRESS: at 67.70% examples, 730107 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:15:09,733 : INFO : EPOCH 5 - PROGRESS: at 68.23% examples, 730325 words/s, in_qsize 1, out_qsize 0
2019-07-06 11:15:10,751 : INFO : EPOCH 5 - PROGRESS: at 68.68% examples, 729504 words/s, in_qsize 9, out_qsize 1
2019-07-06 11:15:11,756 : INFO : EPOCH 5 - PROGRESS: at 69.31% examples, 730408 words/s, in_qsize 0, out_qsize 0
2019-07-06 11:15:12,757 : INFO : EPOCH 5 - PROGRESS: at 69.79% examples, 729908 words/s, in_qsi

2019-07-06 11:16:11,313 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-06 11:16:11,314 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-06 11:16:11,316 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-06 11:16:11,323 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-06 11:16:11,324 : INFO : EPOCH - 5 : training on 173212741 raw words (140583772 effective words) took 191.0s, 735925 effective words/s
2019-07-06 11:16:11,324 : INFO : training on a 866063705 raw words (702925361 effective words) took 956.6s, 734781 effective words/s
2019-07-06 11:16:11,325 : INFO : saving Word2Vec object under models/w2v/model.w2v, separately None
2019-07-06 11:16:11,326 : INFO : storing np array 'vectors' to models/w2v/model.w2v.wv.vectors.npy


FileNotFoundError: [Errno 2] No such file or directory: 'models/w2v/model.w2v.wv.vectors.npy'

In [21]:
model.save('model.w2v')

2019-07-06 11:16:50,199 : INFO : saving Word2Vec object under model.w2v, separately None
2019-07-06 11:16:50,199 : INFO : storing np array 'vectors' to model.w2v.wv.vectors.npy
2019-07-06 11:16:51,949 : INFO : not storing attribute vectors_norm
2019-07-06 11:16:51,950 : INFO : storing np array 'syn1neg' to model.w2v.trainables.syn1neg.npy
C:\Users\USer_IGGr\Anaconda3\envs\Priiincr\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-06 11:16:55,013 : INFO : saved model.w2v
